<a href="https://colab.research.google.com/github/espickle1/boltz-2/blob/main/boltz_2_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install boltz[cuda] -U

In [ ]:
# Upload YAML(s) → enter a compact out_dir → run boltz predict with LIVE output
!pip -q install ipywidgets
from google.colab import output, files
output.enable_custom_widget_manager()

import ipywidgets as W
from IPython.display import display, clear_output
import os, shlex, subprocess, sys

# 1) Upload local YAML(s)
uploaded = files.upload()  # select one or more .yaml files

# 2) Small out_dir input + Run button
out_dir_txt = W.Text(
    value="/content/boltz_results",
    description="out_dir:",
    layout=W.Layout(width="320px")  # compact width
)
run_btn = W.Button(description="Run boltz", button_style="success")
out = W.Output()

def stream_cmd(cmd_list):
    """
    Run a command and stream stdout+stderr line-by-line to the notebook output.
    Returns the process return code.
    """
    proc = subprocess.Popen(
        cmd_list,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,            # decode to str
        bufsize=1,            # line-buffered
        universal_newlines=True
    )
    # Stream live
    for line in proc.stdout:
        print(line, end="")   # do not silence, show as it comes
        sys.stdout.flush()
    return proc.wait()

def on_run(_):
    with out:
        clear_output()
        out_dir = (out_dir_txt.value or "/content/boltz_results").strip()
        os.makedirs(out_dir, exist_ok=True)

        for fname in uploaded.keys():
            print(f"\n=== Running: boltz predict {fname} ===")
            cmd = [
                "boltz", "predict",
                fname,
                "--out_dir", out_dir,
                "--use_msa_server",
            ]
            # Show exact command
            print("$", " ".join(shlex.quote(x) for x in cmd))
            rc = stream_cmd(cmd)
            print(f"\n[boltz exited with code {rc}]")
            if rc != 0:
                print("⚠️ Command failed. Check the logs above.")

display(W.HBox([out_dir_txt, run_btn]))
display(out)
run_btn.on_click(on_run)
